In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [2]:
df2 = pd.read_csv('C:/Users/DMQA/DATA4.csv', encoding = 'euc-kr')

In [3]:
new_df2 = df2.copy()
list_df2 = np.array(df2['review'].tolist())

In [4]:
import re

def cleansing(text):
    cl_text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…/ ]', ' ', text)
    return cl_text

c_list_df2 = []
for x in list_df2 : 
     c_list_df2.append(cleansing(x))

new_df2['review'] = new_df2['review'].apply(cleansing)
new_df2[:10]

,review,ord,ord_group
0,후미권 이끌었고 직선 순위도약 시도나 추입력 발휘안돼 고전,7,0
1,선행으로 경주를 주도했으나 직선막판 근소한 차이로 덜미잡혀,2,1
2,경주전반에 걸쳐 후미권을 벗어나지 못하고 따라가는모습,10,0
3,선두권을 압박하며 따라갔으나 4코너부터 격차벌어지며 고전함,6,0
4,경주전반에 걸쳐 후미권을 벗어나지 못하고 따라가는모습,8,0
5,선행마 후미를 따라가며 기회노렸으나 직선 근소한 차이로 덜미,4,0
6,경주전반에 걸쳐 후미권을 벗어나지 못하고 따라가는모습,11,0
7,초반 선입권 가담이나 2코너이후 순위밀려 직선 후미권으로,10,0
8,경주전반에 걸쳐 후미권을 벗어나지 못하고 따라가는모습,10,0
9,주행심사 경주능력부진 1위마와 30 0 마신차,12,0


In [5]:
from hanspell import spell_checker

def handspell_test(text) :
    spelled_text = spell_checker.check(text)
    handspell_text = spelled_text = spelled_text.checked
    return handspell_text

In [6]:
new_df2['review'] = new_df2['review'].apply(handspell_test)

In [7]:
new_df2

,review,ord,ord_group
0,후미권 이끌었고 직선 순위 도약 시도나 추입력 발휘 안돼 고전,7,0
1,선행으로 경주를 주도했으나 직선 막판 근소한 차이로 덜미잡혀,2,1
2,경주 전반에 걸쳐 후미권을 벗어나지 못하고 따라가는 모습,10,0
3,선두권을 압박하며 따라갔으나 4코너부터 격차 벌어지며 고전함,6,0
4,경주 전반에 걸쳐 후미권을 벗어나지 못하고 따라가는 모습,8,0
...,...,...,...
5227,게이트 극복하며 선행 탄력 유지 우승,1,1
5228,시종 후미 직선 탄력 큰 변화 아님,9,0
5229,선두권 후 미 공략이나 직선 강한 모습을 못 보여줌,6,0
5230,스포츠조선배 후미권에서 순위 좁히지 못해,5,0


In [15]:
train = new_df2.sample(frac=0.8, random_state=42)
test = new_df2.drop(train.index)
print(train.shape)
print(test.shape)

(4186, 3)
(1046, 3)


In [50]:
corpus_df2 = np.array(new_df2['review'].tolist())
print(new_df2[100:110])
print(corpus_df2[100:110])

                                  review  ord  ord_group
100  선행마 후미 출발이나 3 4코너 선입권 이후 직선 순위 더 밀려    7          0
101                 주의  결승선 직선주로 바깥으로 기댐    5          0
102     중위권 전개하며 기회 노렸으나 시종 순위 변동 없이 따라감    5          0
103     선입 최신 전개 펼쳤으나 직선주로 추격 허용해 3착에 그침    3          0
104       선두권을 압박하며 따라갔으나 4코너 이후 지치며 후미로   10          0
105     견책  3코너 진입 지점 안으로 진로 변경 타마 주행 방해    1          1
106      선행으로 경주를 주도했고 시종 선두를 유지하며 우승 성공    1          1
107      중위권 전개하다 직선 추입력을 보여주며 막판 후 착 각축    3          0
108      시종 후미권 전개하다 직선 추입력을 보여주며 후 착 각축    3          0
109      선행으로 경주를 주도했고 시종 선두 유지하며 막판 버티기    1          1
['선행마 후미 출발이나 3 4코너 선입권 이후 직선 순위 더 밀려' '주의  결승선 직선주로 바깥으로 기댐'
 '중위권 전개하며 기회 노렸으나 시종 순위 변동 없이 따라감' '선입 최신 전개 펼쳤으나 직선주로 추격 허용해 3착에 그침'
 '선두권을 압박하며 따라갔으나 4코너 이후 지치며 후미로' '견책  3코너 진입 지점 안으로 진로 변경 타마 주행 방해'
 '선행으로 경주를 주도했고 시종 선두를 유지하며 우승 성공' '중위권 전개하다 직선 추입력을 보여주며 막판 후 착 각축'
 '시종 후미권 전개하다 직선 추입력을 보여주며 후 착 각축' '선행으로 경주를 주도했고 시종 선두 유지하며 막판 버티기']


In [51]:
from soynlp.noun import LRNounExtractor
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

noun_extractor = LRNounExtractor()
nouns = noun_extractor.train_extract(corpus_df2) # list of str like


word_extractor = WordExtractor(
    min_frequency=50, # example
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

word_extractor.train(corpus_df2)
words = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}

noun_scores = {noun:score.score for noun, score in nouns.items()}
combined_scores = {noun:score + cohesion_score.get(noun, 0)
    for noun, score in noun_scores.items()}
combined_scores.update(
    {subword:cohesion for subword, cohesion in cohesion_score.items()
    if not (subword in combined_scores)}
)

tokenizer = LTokenizer(scores=combined_scores)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (785, 465) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 57 nouns are extracted
training was done. used memory 0.394 Gbry 0.394 Gb
all cohesion probabilities was computed. # words = 364
all branching entropies was computed # words = 1512
all accessor variety was computed # words = 1512


In [57]:
train[200:250]

,review,ord,ord_group
4089,후미 전개하다 직선 후 입력 발휘 3착이나 선두와는 격차,3,0
3716,출발은 좋았으나 3 4코너 후미권으로 밀리며 막판 고전,6,0
803,늦출 말 이후 모래 안 맞으려고 바깥 주행,9,0
2699,초반 중위권 가담이나 이후 순위 밀렸고 직선 지친 모습,10,0
485,외곽에서 순위 좁혀 3코너 선두 나선 후 여유 있게 우승,1,1
1215,점프하며 출발로 후미 전개하다 직선 조금 올라옴,7,0
4063,중위권 윤지후 직선 추입력 보이며 신승,1,1
3288,선두권을 압박하며 따라갔으나 직선 지치며 후미로 밀려,8,0
3318,외곽 빠른 출발로 선행마 후미 각축이나 직선 밀려,7,0
373,시종 중간 후미에 경주 내내 큰 변화 보이지 못함,9,0
